<a href="https://colab.research.google.com/github/DarkLord-13/Machine-Learning-01/blob/main/YoutubeSummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Steps:
1. Download youtube video's audio
2. Convert audio to text
3. Perform text summarization

# 1. Download audio file in required format

In [ ]:
!pip install pytube -q

In [2]:
from pytube import YouTube

In [3]:
video_url = 'https://www.youtube.com/watch?v=OLVe2zPq9hI'

In [4]:
yt = YouTube(video_url) # this downloads video specified by url for further processing using pytube (refer documentation)

In [5]:
yt.streams \
  .filter(only_audio=True, file_extension='mp4') \
  .first() \
  .download(filename='ytaudio.mp4')

'/content/ytaudio.mp4'

In [6]:
!ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav # converts the audio to 16000 Hz saves it .wav format

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# 2. English ASR(automatic-speech-recognition) with HuggingSound

speech-to-text

In [7]:
!pip install torch==2.0.1

In [8]:
!pip install huggingsound -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from huggingsound import SpeechRecognitionModel

In [10]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu" # cuda, i.e., gpu if available else cpu

In [11]:
device

'cuda'

In [12]:
# downloading model for english language, other languages also present
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device=device)

INFO:huggingsound.speech_recognition.model:Loading model...


# 3. Audio Chunking

To prevent OUT OF MEMORY error (OOM)

This will prevent utlising the whole RAM by not loading the whole audio at once, rather processing it in chunks of audio.

Here we split the audio into multiple small files and save it seperately.

In [13]:
import librosa

In [14]:
input_file='ytaudio.wav'

you can use silero-VAD to make sure that first the speech content is extracted and then it's splitted into chunks.

this would prevent cutting of the speaker's word into 2 parts.

In [15]:
print(librosa.get_samplerate(input_file))

# stream over 30sec chunks rather than load the full file at once
stream = librosa.stream(
    input_file,
    block_length=30, # 30sec for each chunk
    frame_length=16000,
    hop_length=16000
)

16000


In [16]:
import soundfile as sf

In [17]:
# to save the chunks
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

# Audio Transcription / ASR / Speech-to-Text

In [18]:
audio_path = [] # here we are storing all the audio files we saved, in this list
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [19]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav']

In [20]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 7/7 [00:05<00:00,  1.36it/s]


In [21]:
transcriptions[0]

{'transcription': "tsaying things you know to be untrueat's a goo igdon't lythat's goodbe gratefuland hetry to work for thebest in yourself and other peopleand maybe even broghder try to work for the best despite your syndecism and resentment",
 'start_timestamps': [4160,
  4260,
  4340,
  4380,
  4440,
  4460,
  4480,
  4520,
  4560,
  4580,
  4640,
  4680,
  4720,
  4760,
  4800,
  4820,
  4860,
  4880,
  4920,
  4940,
  4960,
  5000,
  5040,
  5060,
  5100,
  5140,
  5180,
  5200,
  5260,
  5300,
  5380,
  5420,
  5520,
  5600,
  5720,
  5760,
  7860,
  7920,
  7940,
  7960,
  7980,
  8000,
  8020,
  8060,
  8080,
  8140,
  8200,
  8340,
  8440,
  8760,
  8820,
  8860,
  8900,
  8920,
  8960,
  9000,
  9180,
  11380,
  11400,
  11440,
  11500,
  11520,
  11540,
  11580,
  11640,
  11680,
  11760,
  11800,
  16100,
  16180,
  16220,
  16240,
  16300,
  16380,
  16440,
  16460,
  16520,
  16580,
  16600,
  19500,
  19540,
  19560,
  19600,
  19660,
  19700,
  20500,
  20580,
  20660,


In [22]:
full_transcript = ''
for item in transcriptions:
  full_transcript+=''.join(item['transcription'])

In [23]:
full_transcript

"tsaying things you know to be untrueat's a goo igdon't lythat's goodbe gratefuland hetry to work for thebest in yourself and other peopleand maybe even broghder try to work for the best despite your syndecism and resentmenti suppose t's a form of gratitude as wellthat's what love means fundamentally is working for the best of things it's a goal what's the alternative t's not can make things betters can make things worse younogenerally speas like truth nested inside  love then there's gratitude lorking around the edges beauts importantriye wet more beauty period everywhere that would be niceit's kind of intimidating beautywhat doyou mean bywell because it's an idealat judges youis something that's quite interesting every ideal is a judgebecause i nalort ofyea but it's interesting it's an interesting problem tatyou need an ideal towork towards the consequence of tat is your judged by it thats fascinating because we always want the ideal rightye i recommended to people that they try to r

In [24]:
from transformers import pipeline

In [25]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [26]:
summarized_text = summarization(full_transcript)

In [27]:
summarized_text[0]['summary_text']

" Love means fundamentally is working for the best of things it's a goal what's the alternative t's not can make things betters or make things worse younogenerally speas like truth nested inside  love then there's gratitude lorking around the edges ."

Text Chunking before Summarization

In [28]:
'''
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  #print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
 # print("Summarized text\n"+out)
  summarized_text.append(out)

#print(summarized_text)
'''

'\nnum_iters = int(len(full_transcript)/1000)\nsummarized_text = []\nfor i in range(0, num_iters + 1):\n  start = 0\n  start = i * 1000\n  end = (i + 1) * 1000\n  #print("input text \n" + full_transcript[start:end])\n  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)\n  out = out[0]\n  out = out[\'summary_text\']\n # print("Summarized text\n"+out)\n  summarized_text.append(out)\n\n#print(summarized_text)\n'